# Using python packages with your programs

In this document, we will learn how to install custom dependencies to your program.

Let's create another file with our new program [./source_files/program_with_dependencies.py](./source_files/program_with_dependencies.py). 

For the sake of this example, let's use the `qiskit-experiments` package as our custom dependency. We will use randomized benchmarking (RB) circuits from `qiskit-experiments`, composed with the circuit from the input arguments for measurement.

Here's what the file would look like:

```python
# source_files/program_with_dependencies.py

from quantum_serverless import get_arguments, save_result

from qiskit.primitives import Sampler
from qiskit_experiments.library import StandardRB


arguments = get_arguments()

circuit = arguments.get("circuit")

rb = StandardRB(
    physical_qubits=(1,),
    lengths=list(range(1, 300, 30)),
    seed=42
)
composed = circuit.compose(rb.circuits()[0])

sampler = Sampler()

quasi_dists = sampler.run(composed).result().quasi_dists

print(f"Quasi distribution: {quasi_dists[0]}")

# saving results of a program
save_result({
    "quasi_dists": quasi_dists[0]
})

```

As you can see, we've imported our custom dependency, `qiskit-experiments`, and used its `StandardRB` module to generate an RB circuit, which we've composed with the circuit provided in the input arguments.

Now, let's create and configure our client

To install a custom dependency that our program might use we need to pass it as the `dependencies` argument to the [Program](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.Program.html#quantum_serverless.core.Program) class constructor. 
You can pass multiple dependencies and specify versions. 

In [1]:
from quantum_serverless import Program

program = Program(
    title="Program with dependencies",
    entrypoint="program_with_dependencies.py",
    working_dir="./source_files/",
    dependencies=["qiskit-experiments==0.5.2"],
)

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit-extensions.github.io/quantum-serverless/deployment/local.html).

In [2]:
from quantum_serverless import QuantumServerless, Provider
import os

provider = Provider(
    username=os.environ.get("GATEWAY_USER", "user"),
    password=os.environ.get("GATEWAY_PASSWORD", "password123"),
    # token=os.environ.get("GATEWAY_TOKEN", "<TOKEN>"), # token can be used instead of user/password combination
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

In [3]:
from qiskit.circuit.random import random_circuit

circuit = random_circuit(2, 2)

In [4]:
job = serverless.run(program, arguments={"circuit": circuit})

In [5]:
job.status()

'QUEUED'

In [6]:
job.result()

'{"quasi_dists": {"0": 0.6107159035567002, "1": 0.3892840964432998}}'